In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csc_matrix
import umap
from sklearn.preprocessing import StandardScaler
import random
from scipy.stats.stats import pearsonr
random.seed(123)
from joblib import Parallel, delayed

/tmp/ipykernel_19443/377559581.py:15: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


In [2]:
meta_data=pd.read_csv('/home//jupyter/BRI_Analysis/scRNA/hise_meta_data_2024-01-23_fixed.csv')

In [4]:
meta_data['subject.subjectGuid'].value_counts().head(10)

subject.subjectGuid
BR2034    10
BR2014    10
BR2044    10
BR2012    10
BR1050    10
BR2032    10
BR1030    10
BR2048    10
BR2047    10
BR1049    10
Name: count, dtype: int64

In [5]:
donor=['BR1050','BR1030','BR2014','BR2034']

In [6]:
meta_subset=meta_data[meta_data['subject.subjectGuid'].isin(donor)]

In [8]:
from tqdm import tqdm
h5_list = []
for i in tqdm(range(len(meta_subset["pbmc_sample_id"].tolist()))):
    try:
        result = sc.read_h5ad("/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/"+meta_subset["pbmc_sample_id"].tolist()[i]+'.h5ad')
        h5_list.append(result)
        print()
    except Exception as e:
        print(f'Error reading {file_name}: {e}')

  2% 1/40 [00:02<01:28,  2.27s/it]

  5% 2/40 [00:04<01:26,  2.27s/it]

  8% 3/40 [00:06<01:16,  2.07s/it]

 10% 4/40 [00:08<01:17,  2.15s/it]

 12% 5/40 [00:10<01:12,  2.07s/it]

...

 90% 36/40 [01:10<00:06,  1.65s/it]

 92% 37/40 [01:11<00:04,  1.58s/it]

 95% 38/40 [01:12<00:02,  1.32s/it]

 98% 39/40 [01:14<00:01,  1.35s/it]

100% 40/40 [01:14<00:00,  1.87s/it]

In [9]:
%%time
combined= anndata.concat(h5_list)

CPU times: user 1.89 s, sys: 1.39 s, total: 3.27 s
Wall time: 3.27 s


In [10]:
combined.obs.index=combined.obs['barcodes']

In [11]:
sc.pp.normalize_total(combined, target_sum=1e4)
sc.pp.log1p(combined)

In [13]:
combined.write_h5ad('visualization_2BR1_2BR2_10TP_test_data.h5ad')
